In [1]:
# ===============================================================
# CELL 1: Setup, Imports, and Helper Functions
# ===============================================================

import re
import socket
from urllib.parse import urlparse
import tldextract
import requests

# --- Tokenize URL into "words" for word-level statistics
def tokenize_url_words(url: str):
    lowered = url.lower()
    cleaned = re.sub(r'[/:?=&\.\-_~%\+]+', ' ', lowered)
    words = [w for w in cleaned.split() if w]
    return words

# --- Extract key URL parts (scheme, host, domain, path, etc.)
def parse_url_bits(url: str):
    parsed = urlparse(url)

    scheme = parsed.scheme
    netloc = parsed.netloc
    path   = parsed.path or ""
    query  = parsed.query or ""

    base_url = f"{scheme}://{netloc}"
    host_only = netloc.split('@')[-1]
    host_no_port = host_only.split(':')[0]

    tld_info = tldextract.extract(url)
    subdomain = tld_info.subdomain or ""
    domain    = tld_info.domain or ""
    suffix    = tld_info.suffix or ""

    full_path_q = path + ("?" + query if query else "")
    words_raw = tokenize_url_words(url)

    return {
        "scheme": scheme,
        "netloc": netloc,
        "host": host_no_port,
        "domain": domain,
        "subdomain": subdomain,
        "suffix": suffix,
        "path": path,
        "query": query,
        "path_plus_query": full_path_q,
        "base_url": base_url,
        "words_raw": words_raw,
        "url": url
    }

# --- Safe digit ratio helper
def safe_ratio_digits(s: str):
    if len(s) == 0:
        return 0
    return len(re.sub("[^0-9]", "", s)) / len(s)

# --- Brand check in subdomain (requires allbrands.txt)
with open("data/allbrands.txt", "r") as f:
    allbrand = [line.strip().lower() for line in f if line.strip()]

def brand_in_subdomain(domain_str, subdomain_str):
    for b in allbrand:
        if b in subdomain_str and b != domain_str:
            return 1
    return 0


In [2]:
# ===============================================================
# CELL 2: Group 1 — Basic URL Structure / Syntax
# ===============================================================

def extract_group1_basic(url: str): 
    parts = parse_url_bits(url)

    # def has_ip(u):
    #     pattern = re.compile(r'(\d{1,3}\.){3}\d{1,3}')
    #     return 1 if pattern.search(u) else 0
    

    def has_ip(url):
        match = re.search(
            '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
            '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
            '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)|'  # IPv4 in hexadecimal
            '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|'
            '[0-9a-fA-F]{7}', url)  # Ipv6
        if match:
            return 1
        else:
            return 0

    def has_prefix_suffix(u):
        return 1 if re.search(r"-", urlparse(u).netloc.split('.')[-2]) else 0

    def has_punycode(u):
        return 1 if "xn--" in u else 0

    def has_tld_in_path(tld, path):
        return 1 if tld and tld in path else 0

    def has_tld_in_subdomain(tld, subdomain):
        return 1 if tld and tld in subdomain else 0

    def abnormal_subdomain(u):
        sub = urlparse(u).netloc.split('.')
        return 1 if len(sub) > 3 else 0

    return {
        "length_url": len(url),
        "length_hostname": len(parts["host"]),
        "ip": has_ip(url),
        "nb_dots": parts["host"].count('.'),
        "port": 1 if ":" in parts["netloc"] else 0,
        "ratio_digits_url": safe_ratio_digits(parts["url"]),
        "ratio_digits_host": safe_ratio_digits(parts["host"]),
        "punycode": has_punycode(url),
        "nb_subdomains": len(parts["subdomain"].split('.')) if parts["subdomain"] else 0,
        "prefix_suffix": has_prefix_suffix(url),
        "shortening_service": 1 if re.search(r"(bit\.ly|goo\.gl|tinyurl\.com|t\.co|ow\.ly)", url) else 0,
        "tld_in_path": has_tld_in_path(parts["suffix"], parts["path_plus_query"]),
        "tld_in_subdomain": has_tld_in_subdomain(parts["suffix"], parts["subdomain"]),
        "abnormal_subdomain": abnormal_subdomain(url),
        "nb_dslash": url.count("//") - 1,
        "http_in_path": 1 if "http" in parts["path_plus_query"] else 0,
        "https_token": 1 if "https" in parts["scheme"] else 0
    }


In [3]:
# ===============================================================
# CELL 3: Group 2 — Symbol Counts and Redirects
# ===============================================================

def extract_group2_chars(url: str):
    parts = parse_url_bits(url)

    # attempt HTTP GET for redirect info
    try:
        page = requests.get(url, allow_redirects=True, timeout=3)
    except Exception:
        class Dummy:
            history = []
        page = Dummy()

    def count_symbol(u, s): return u.count(s)
    def count_double_slash(u): return u.count("//") - 1

    nb_redirection = len(page.history)
    nb_external_redirection = sum(
        1 for r in page.history if parts["domain"].lower() not in r.url.lower()
    )

    return {
        "nb_hyphens": count_symbol(url, "-"),
        "nb_at": count_symbol(url, "@"),
        "nb_qm": count_symbol(url, "?"),
        "nb_and": count_symbol(url, "&"),
        "nb_or": url.lower().count("|"),
        "nb_eq": count_symbol(url, "="),
        "nb_underscore": count_symbol(url, "_"),
        "nb_tilde": count_symbol(url, '~'),
        
#################################################################################################################################
#              Checks if tilde symbol exist in webpage URL 
#################################################################################################################################

# def count_tilde(full_url):
#     if full_url.count('~')>0:
#         return 1
#     return 0
    
        "nb_percent": count_symbol(url, "%"),
        "nb_slash": count_symbol(url, "/"),
        "nb_star": count_symbol(url, "*"),
        "nb_colon": count_symbol(url, ":"),
        "nb_comma": count_symbol(url, ","),
        "nb_semicolumn": count_symbol(url, ";"),
        "nb_dollar": count_symbol(url, "$"),
        "nb_space": count_symbol(url, " "),
        "nb_www": 1 if "www" in url.lower() else 0,
        "nb_com": url.lower().count(".com"),
        "nb_dslash": count_double_slash(url),
        "nb_redirection": nb_redirection,
        "nb_external_redirection": nb_external_redirection,
    }


In [4]:
# ===============================================================
# CELL 4: Group 3 — Word Statistics
# ===============================================================

def extract_group3_wordstats(url: str):
    parts = parse_url_bits(url)
    words = parts["words_raw"]

    def length_words_raw(words): return sum(len(w) for w in words)
    def char_repeat(words): return max([words.count(w) for w in set(words)]) if words else 0
    def shortest(words): return min(map(len, words)) if words else 0
    def longest(words): return max(map(len, words)) if words else 0
    def average(words): return sum(map(len, words)) / len(words) if words else 0

    host = [parts["host"]]
    path = [parts["path"] or ""]

    return {
        "length_words_raw": length_words_raw(words),
        "char_repeat": char_repeat(words),
        "shortest_words_raw": shortest(words),
        "shortest_word_host": shortest(host),
        "shortest_word_path": shortest(path),
        "longest_words_raw": longest(words),
        "longest_word_host": longest(host),
        "longest_word_path": longest(path),
        "avg_words_raw": average(words),
        "avg_word_host": average(host),
        "avg_word_path": average(path),
    }


In [5]:
# ===============================================================
# CELL 5: Group 4 — Phishing / Brand / DNS / TLD Heuristics
# ===============================================================

def extract_group4_phish(url: str):
    parts = parse_url_bits(url)

    def phish_hints(path_q):
        hints = ['wp', 'login', 'includes', 'admin', 'content', 'site', 'images', 'js', 'alibaba', 'css', 'myaccount', 'dropbox', 'themes', 'plugins', 'signin', 'view']
        return 1 if any(h in path_q.lower() for h in hints) else 0

    def domain_in_brand(domain):
        return 1 if domain in allbrand else 0

    def brand_in_path(domain, path_q):
        for b in allbrand:
            if b in path_q and b != domain:
                return 1
        return 0

    def suspecious_tld(tld):
        bad_tlds = ["zip", "xyz", "top", "gq", "tk", "ml", "cf"]
        return 1 if tld in bad_tlds else 0

    def statistical_report(url, domain):
        url_match = re.search('at\.ua|usa\.cc|beget\.tech|16mb\.com', url)
        try:
            ip_address = socket.gethostbyname(domain)
            ip_match = re.search('146\.112\.61\.108|23\.253\.164\.', ip_address)
            if url_match or ip_match:
                return 1
        except:
            pass
        return 0

    return {
        "phish_hints": phish_hints(parts["path_plus_query"]),
        "domain_in_brand": domain_in_brand(parts["domain"]),
        "brand_in_subdomain": brand_in_subdomain(parts["domain"], parts["subdomain"]),
        "brand_in_path": brand_in_path(parts["domain"], parts["path_plus_query"]),
        "suspecious_tld": suspecious_tld(parts["suffix"]),
        "statistical_report": statistical_report(parts["url"], parts["domain"]),
    }


<>:26: SyntaxWarning: invalid escape sequence '\.'
<>:29: SyntaxWarning: invalid escape sequence '\.'
<>:26: SyntaxWarning: invalid escape sequence '\.'
<>:29: SyntaxWarning: invalid escape sequence '\.'
C:\Users\khyal\AppData\Local\Temp\ipykernel_2456\683812489.py:26: SyntaxWarning: invalid escape sequence '\.'
  url_match = re.search('at\.ua|usa\.cc|beget\.tech|16mb\.com', url)
C:\Users\khyal\AppData\Local\Temp\ipykernel_2456\683812489.py:29: SyntaxWarning: invalid escape sequence '\.'
  ip_match = re.search('146\.112\.61\.108|23\.253\.164\.', ip_address)


In [12]:
# ===============================================================
# CELL 6: Combine All Feature Groups into One Extractor
# ===============================================================



import re
from collections import OrderedDict


# ===============================================================
# NLP helper function (used for random_domain feature)
# ===============================================================
def check_word_random(domain: str) -> int:
    """
    Minimal heuristic for detecting random-looking domains.
    Returns 1 if the domain seems random, else 0.
    """
    d = domain.lower()

    # Heuristic 1: contains digits
    if re.search(r"\d", d):
        return 1

    # Heuristic 2: too many consonants in a row (>=4)
    if re.search(r"[bcdfghjklmnpqrstvwxyz]{4,}", d):
        return 1

    # Looks normal
    return 0


# ===============================================================
# Combine All Feature Groups into One Extractor
# ===============================================================
def extract_all_url_structure_features(url: str):
    g1 = extract_group1_basic(url)
    g2 = extract_group2_chars(url)
    g3 = extract_group3_wordstats(url)
    g4 = extract_group4_phish(url)

    parts = parse_url_bits(url)

    # Use the local function instead of a class
    random_val = check_word_random(parts["domain"])

    features = OrderedDict([
        ("length_url", g1["length_url"]),
        ("length_hostname", g1["length_hostname"]),
        ("ip", g1["ip"]),
        ("nb_dots", g1["nb_dots"]),
        ("nb_hyphens", g2["nb_hyphens"]),
        ("nb_at", g2["nb_at"]),
        ("nb_qm", g2["nb_qm"]),
        ("nb_and", g2["nb_and"]),
        ("nb_or", g2["nb_or"]),
        ("nb_eq", g2["nb_eq"]),
        ("nb_underscore", g2["nb_underscore"]),
        ("nb_tilde", g2["nb_tilde"]),
        ("nb_percent", g2["nb_percent"]),
        ("nb_slash", g2["nb_slash"]),
        ("nb_star", g2["nb_star"]),
        ("nb_colon", g2["nb_colon"]),
        ("nb_comma", g2["nb_comma"]),
        ("nb_semicolumn", g2["nb_semicolumn"]),
        ("nb_dollar", g2["nb_dollar"]),
        ("nb_space", g2["nb_space"]),
        ("nb_www", g2["nb_www"]),
        ("nb_com", g2["nb_com"]),
        ("nb_dslash", g1["nb_dslash"]),
        ("http_in_path", g1["http_in_path"]),
        ("https_token", g1["https_token"]),
        ("ratio_digits_url", g1["ratio_digits_url"]),
        ("ratio_digits_host", g1["ratio_digits_host"]),
        ("punycode", g1["punycode"]),
        ("port", g1["port"]),
        ("tld_in_path", g1["tld_in_path"]),
        ("tld_in_subdomain", g1["tld_in_subdomain"]),
        ("abnormal_subdomain", g1["abnormal_subdomain"]),
        ("nb_subdomains", g1["nb_subdomains"]),
        ("prefix_suffix", g1["prefix_suffix"]),
        ("random_domain", random_val),
        ("shortening_service", g1["shortening_service"]),
        ("path_extension", 1 if "." in parts["path"].split("/")[-1] else 0),
        ("nb_redirection", g2["nb_redirection"]),
        ("nb_external_redirection", g2["nb_external_redirection"]),
        ("length_words_raw", g3["length_words_raw"]),
        ("char_repeat", g3["char_repeat"]),
        ("shortest_words_raw", g3["shortest_words_raw"]),
        ("shortest_word_host", g3["shortest_word_host"]),
        ("shortest_word_path", g3["shortest_word_path"]),
        ("longest_words_raw", g3["longest_words_raw"]),
        ("longest_word_host", g3["longest_word_host"]),
        ("longest_word_path", g3["longest_word_path"]),
        ("avg_words_raw", g3["avg_words_raw"]),
        ("avg_word_host", g3["avg_word_host"]),
        ("avg_word_path", g3["avg_word_path"]),
        ("phish_hints", g4["phish_hints"]),
        ("domain_in_brand", g4["domain_in_brand"]),
        ("brand_in_subdomain", g4["brand_in_subdomain"]),
        ("brand_in_path", g4["brand_in_path"]),
        ("suspecious_tld", g4["suspecious_tld"]),
        ("statistical_report", g4["statistical_report"]),
    ])

    return features


In [21]:
# # ===============================================================
# # CELL 7: Testing the Extractor
# # ===============================================================

# test_urls = [
#     "https://support-appleld.com.secureupdate.duilawyeryork.com/ap/89e6a3b4b063b8d/?cmd=_update&dispatch=89e6a3b4b063b8d1b&locale=_"
# ]

# for u in test_urls:
#     print("\nURL:", u)
#     features = extract_all_url_structure_features(u)
#     print(f"Extracted {len(features)} features:")
#     features
# features


In [40]:
# ===============================================================
# CELL 7: Testing the Extractor
# ===============================================================

test_urls = [
    "http://rgipt.ac.in"
]

results = {}

for u in test_urls:
    print("\nURL:", u)
    features = extract_all_url_structure_features(u)
    features_dict = dict(features)  # convert OrderedDict → normal dict
    print(f"Extracted {len(features_dict)} features.\n")
    results[u] = features_dict  # store result in a dictionary

# Now you can access the result like:
feature_input = results[test_urls[0]]  # features for the first test URL
feature_input


URL: http://rgipt.ac.in
Extracted 56 features.



{'length_url': 18,
 'length_hostname': 11,
 'ip': 0,
 'nb_dots': 2,
 'nb_hyphens': 0,
 'nb_at': 0,
 'nb_qm': 0,
 'nb_and': 0,
 'nb_or': 0,
 'nb_eq': 0,
 'nb_underscore': 0,
 'nb_tilde': 0,
 'nb_percent': 0,
 'nb_slash': 2,
 'nb_star': 0,
 'nb_colon': 1,
 'nb_comma': 0,
 'nb_semicolumn': 0,
 'nb_dollar': 0,
 'nb_space': 0,
 'nb_www': 0,
 'nb_com': 0,
 'nb_dslash': 0,
 'http_in_path': 0,
 'https_token': 0,
 'ratio_digits_url': 0.0,
 'ratio_digits_host': 0.0,
 'punycode': 0,
 'port': 0,
 'tld_in_path': 0,
 'tld_in_subdomain': 0,
 'abnormal_subdomain': 0,
 'nb_subdomains': 0,
 'prefix_suffix': 0,
 'random_domain': 0,
 'shortening_service': 0,
 'path_extension': 0,
 'nb_redirection': 1,
 'nb_external_redirection': 0,
 'length_words_raw': 13,
 'char_repeat': 1,
 'shortest_words_raw': 2,
 'shortest_word_host': 11,
 'shortest_word_path': 0,
 'longest_words_raw': 5,
 'longest_word_host': 11,
 'longest_word_path': 0,
 'avg_words_raw': 3.25,
 'avg_word_host': 11.0,
 'avg_word_path': 0.0,
 'phish_

# Fetaure iput and model response

In [41]:
import numpy as np
import pandas as pd
import joblib

def predict_url_features(feature_dict_or_series, artifacts_path="phish_artifacts.pkl"):
    # Load artifacts
    bundle = joblib.load(artifacts_path)
    selected_cols = bundle["selected_cols"]
    scaler       = bundle["scaler"]
    best_model   = bundle["best_model"]
    best_scaled  = bundle["best_scaled"]
    name         = bundle["best_model_name"]
 
    # Build one-row DataFrame from the extracted features (56 keys)
    X_new = pd.DataFrame([dict(feature_dict_or_series)])

    # Align to training-time selection & order:
    # - keep only the selected columns
    # - add any missing ones as 0 (or another default if you prefer)
    X_new = X_new.reindex(columns=selected_cols, fill_value=0)

    # Apply the same scaler if the model expects scaled inputs
    if best_scaled:
        X_new_arr = scaler.transform(X_new)
    else:
        X_new_arr = X_new.values

    # Predict label
    y_pred = best_model.predict(X_new_arr)[0]

    # Probability if supported
    try:
        y_proba = best_model.predict_proba(X_new_arr)[:, 1][0]
    except Exception:
        y_proba = (best_model.decision_function(X_new_arr)[0]
                   if hasattr(best_model, "decision_function") else np.nan)

    return {
        "model": name,
        "label": y_pred,
        "phishing_probability": float(y_proba) if np.isscalar(y_proba) else np.nan
    }


In [ ]:
# new_features = {
#     'length_url': 126, 'length_hostname': 50, 'ip': 1, 'nb_dots': 4, 'nb_hyphens': 1, 'nb_at': 0,
#     'nb_qm': 1, 'nb_and': 2, 'nb_or': 0, 'nb_eq': 3, 'nb_underscore': 2, 'nb_tilde': 0, 'nb_percent': 0,
#     'nb_slash': 5, 'nb_star': 0, 'nb_colon': 1, 'nb_comma': 0, 'nb_semicolumn': 0, 'nb_dollar': 0,
#     'nb_space': 0, 'nb_www': 0, 'nb_com': 2, 'nb_dslash': 0, 'http_in_path': 0, 'https_token': 1,
#     'ratio_digits_url': 0.15079365079365079, 'ratio_digits_host': 0.0, 'punycode': 0, 'port': 0,
#     'tld_in_path': 0, 'tld_in_subdomain': 1, 'abnormal_subdomain': 1, 'nb_subdomains': 3, 'prefix_suffix': 0,
#     'random_domain': 0, 'shortening_service': 0, 'path_extension': 0, 'nb_redirection': 0,
#     'nb_external_redirection': 0, 'length_words_raw': 107, 'char_repeat': 2, 'shortest_words_raw': 2,
#     'shortest_word_host': 50, 'shortest_word_path': 20, 'longest_words_raw': 17, 'longest_word_host': 50,
#     'longest_word_path': 20, 'avg_words_raw': 7.642857142857143, 'avg_word_host': 50.0, 'avg_word_path': 20.0,
#     'phish_hints': 1, 'domain_in_brand': 0, 'brand_in_subdomain': 1, 'brand_in_path': 0,
#     'suspecious_tld': 0, 'statistical_report': 0
# }
new_features = feature_input  # Use the features extracted earlier
result = predict_url_features(new_features)
print(result)


{'model': 'Random Forest', 'label': 'legitimate', 'phishing_probability': 0.3358815643420907}


c:\Users\khyal\Desktop\Phishing Detection System\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\khyal\Desktop\Phishing Detection System\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [43]:
result

{'model': 'Random Forest',
 'label': 'legitimate',
 'phishing_probability': 0.3358815643420907}

In [46]:
required_result = {
    'label': result['label'],
    'phishing_probability': round(result['phishing_probability'], 2)
}
required_result

{'label': 'legitimate', 'phishing_probability': 0.34}

In [ ]:
{'label': 'legitimate',
 'phishing_probability': 0.33}

{'label': 'legitimate', 'phishing_probability': 0.34}

In [ ]:
0 i want to reduce latency - i will look for more efficient libraries
1 additional features - scraping
2 additional model training - based on new feature we will do ensamle modeling
3 look for existing datasets with all phishing urls and legit urls but might have different features. then retrain model
